In [1]:
import sys
import os
import math
import copy
import json
import random

In [12]:
class Matrix:
    def __init__(self, nrows, ncols, init='zeros'):
        assert nrows > 0 and ncols > 0
        self.nrows = nrows
        self.ncols = ncols
        
        if init == 'zeros':
            self.data = [[0 for col in range(ncols)] for row in range(nrows)]
        elif init == 'ones':
            self.data = [[1 for col in range(ncols)] for row in range(nrows)]
        elif init == 'random':
            self.data = [[random.random() for col in range(ncols)] for row in range(nrows)]
        elif init == 'eye':
            self.data = [[1 if col == row else 0 for col in range(ncols)] for row in range(nrows)]

    @staticmethod
    def fromDict(data):
        ncols = data['ncols']
        nrows = data['nrows']
        items = data['data']
        assert len(items) == ncols * nrows
        m = Matrix(nrows, ncols)
        for row in range(nrows):
            for col in range(ncols):
                m[(row, col)] = items[ncols * row + col]
        return m

    @staticmethod
    def toDict(M):
        assert isinstance(M, Matrix)
        nrows, ncols = M.shape()
        data = []
        for row in range(nrows):
            for col in range(ncols):
                data.append(M[(row, col)])
        return {'nrows': nrows, 'ncols': ncols, 'data': data}

    def __str__(self):
        return '\n'.join(['\t'.join([str(elem) for elem in row]) for row in self.data])

    def __repr__(self):
        return repr(self.data)

    def shape(self):
        return self.nrows, self.ncols

    def __getitem__(self, index):
        row, col = index
        return self.data[row][col]

    def __setitem__(self, index, value):
        row, col = index
        self.data[row][col] = value

    def __sub__(self, rhs):
        assert self.nrows == rhs.nrows and self.ncols == rhs.ncols
        res = [[self.data[row][col] - rhs.data[row][col] for col in range(self.ncols)] for row in range(self.nrows)]
        
        new_matrix = Matrix(self.nrows, self.ncols)
        new_matrix.data = copy.deepcopy(res)
        
        return new_matrix

    def __add__(self, rhs):
        assert self.nrows == rhs.nrows and self.ncols == rhs.ncols
        res = [[self.data[row][col] + rhs.data[row][col] for col in range(self.ncols)] for row in range(self.nrows)]
        
        new_matrix = Matrix(self.nrows, self.ncols)
        new_matrix.data = copy.deepcopy(res)
        
        return new_matrix
    
    def __mul__(self, rhs):
        assert self.ncols == rhs.nrows
        res = [[sum(self.data[row][cur] * rhs.data[cur][col] for cur in range(rhs.nrows)) for col in range(rhs.ncols)] 
               for row in range(self.nrows)]
        
        new_matrix = Matrix(self.nrows, rhs.ncols)
        new_matrix.data = copy.deepcopy(res)
        
        return new_matrix
    
    def __pow__(self, power):
        res = [[self.data[row][col] ** power for col in range(self.ncols)] for row in range(self.nrows)]
        
        new_matrix = Matrix(self.nrows, self.ncols)
        new_matrix.data = copy.deepcopy(res)
        
        return new_matrix

    def sum(self):
        return sum(sum(self.data[row][col] for row in range(self.nrows)) for col in range(self.ncols))

    def minor(self, remove_row, remove_col):
        minor = [row[:remove_col] + row[remove_col + 1:] for row in (self.data[:remove_row] + self.data[remove_row + 1:])]
        
        minor_matrix = Matrix(self.nrows - 1, self.ncols - 1)
        minor_matrix.data = copy.deepcopy(minor)
        
        return minor_matrix
    
    def det(self):
        assert self.nrows == self.ncols
        if self.nrows == 2:
            return self.data[0][0] * self.data[1][1] - self.data[0][1] * self.data[1][0]
        else:
            return sum((-1) ** col * self.data[0][col] * self.minor(0, col).det() for col in range(self.ncols))
    
    def transpose(self):
        res = [[self.data[row][col] for row in range(self.nrows)] for col in range(self.ncols)]
        
        new_matrix = Matrix(self.nrows, self.ncols)
        new_matrix.data = copy.deepcopy(res)
        
        return new_matrix

    def inv(self):
        assert self.nrows == self.ncols
        det = self.det()
        assert det != 0
        
        if self.nrows == 2:
            res = [[self.data[1][1] / det, -1 * self.data[0][1] / det], 
                   [-1 * self.data[1][0] / det, self.data[0][0] / det]]
            
            new_matrix = Matrix(self.nrows, self.ncols)
            new_matrix.data = copy.deepcopy(res)
            
            return new_matrix

        else:
            minors_data = [[((-1) ** (row + col)) * self.minor(row, col).det() for col in range(self.ncols)] 
                           for row in range(self.nrows)]
            
            minors = Matrix(self.nrows, self.ncols)
            minors.data = copy.deepcopy(minors_data)
            
            minors_t = minors.transpose()
            minors_t_data = [[minors_t.data[row][col] / det for col in range(self.ncols)] for row in range(self.nrows)]
            minors_t.data = copy.deepcopy(minors_t_data)
            
            return minors_t

In [13]:
def load_file(filename):
    with open(filename, 'r') as f:
        input_file = json.load(f)
        A = Matrix.fromDict(input_file['A'])
        B = Matrix.fromDict(input_file['B'])
    return A, B

In [14]:
filename = 'input_007.json'
A, B = load_file(filename)
print('Матрица A: ')
print(A)
print('Матрица B: ')
print(B)
C = A*B
print('Матрица C = A*B: ')
print(C)
C_t = C.transpose()
print('Транспонированная матрица C: ')
print(C_t)
C_inv = C.inv()
print('Матрица, обратная C: ')
print(C_inv)
E = Matrix(C_inv.ncols, C_inv.nrows, init='eye')
D = C_inv + E
print('Матрица D равная сумме C и единичной матрицы: ')
print(D)
D_det = D.det()
print('Определитель матрицы D: ', D_det)
D_norm = (D**2).sum()**0.5
print('Норма Фробениуса матрицы D: ', D_norm)

Матрица A: 
0.8664507264366117	0.8711172185977024	0.8388579010345589	0.3385954863376641
0.5755175536659002	0.18678610650634497	0.05276305229332012	0.670699458779274
0.1971662046816679	0.9988933959766749	0.6228840826224107	0.6773445844538808
Матрица B: 
0.5966599094269356	0.806899427629966	0.28363432482428075
0.738604444938575	0.7320805344250513	0.8616089381122172
0.6568308929811251	0.09722808436751229	0.7231867338381839
0.8842057816391085	0.23461916011957917	0.3070971071838128
Матрица C = A*B: 
2.010763332554833	1.4978680895681062	1.7069501484369205
1.10904204191135	0.7636162515296646	0.5683005146758001
1.8634697780642444	1.1098435525183763	1.5750506492617236
Транспонированная матрица C: 
2.010763332554833	1.10904204191135	1.8634697780642444
1.4978680895681062	0.7636162515296646	1.1098435525183763
1.7069501484369205	0.5683005146758001	1.5750506492617236
Матрица, обратная C: 
-2.750518388842292	2.234857170702184	2.174487082927301
3.307233881623085	0.06633793551129148	-3.6081273009915784

In [18]:
A_dict = Matrix.toDict(A)
B_dict = Matrix.toDict(B)
C_dict = Matrix.toDict(C)
Ct_dict = Matrix.toDict(C_t)
Cinv_dict = Matrix.toDict(C_inv)
result = {
    'A': A_dict,
    'B': B_dict,
    'C': C_dict,
    'Ct': Ct_dict,
    'Cinv': Cinv_dict,
    'D_det': D_det,
    'D_norm': D_norm
}

In [19]:
def save_file(filename, data):
    with open(filename, 'w') as f:
        input_file = json.dump(data, f)

In [20]:
save_file('result.json', result)